In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [1]:
import pandas as pd
import numpy as np

path =('../../dataset/')

# 장르 결측치 처리 된 거 이후부터 시작!
movies_data = pd.read_csv('movies_genre_crawling.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [184]:
movie_dataset = movies_data.copy()

In [3]:
links_data = pd.read_csv(path+'links.csv')

In [4]:
movies = pd.merge(movie_dataset, links_data, on='movieId')

In [5]:
def create_imdb_url(imdb_id):
    # imdbId를 문자열로 변환하고, 7자리가 되도록 앞에 0을 추가
    imdb_id_str = str(imdb_id).zfill(7)
    # URL 생성
    url = f"https://www.imdb.com/title/tt{imdb_id_str}/"
    return url

# 크롤링 고

In [186]:
browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

names = movie_dataset['title'].tolist()

In [190]:
from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]
for name in tqdm(names[3000: 4000], desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(3.5)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    time.sleep(1.5)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movie_dataset.loc[movie_dataset['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movie_dataset.loc[movie_dataset['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movie_dataset.loc[movie_dataset['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        
    ################################################################################
    body.send_keys(Keys.PAGE_UP)
    body.send_keys(Keys.PAGE_UP)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1.5)
    ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')
        movie_dataset.loc[movie_dataset['title'] == name, 'age_rating'] = age_rating[0].text

    except:
        un_age_rating.append(name)
        
     ################################################################################
    # tags
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_1'] = tags[0].text
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_2'] = tags[1].text
        movie_dataset.loc[movie_dataset['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)

    ################################################################################
 
    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movie_dataset.loc[movie_dataset['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
   
    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|████████████████████████████████████████████████████████| 1000/1000 [3:40:34<00:00, 13.23s/movie]


In [188]:
# null 값 확인
print("storyline : ", len(un_storyline))
print("----------"*10)
print("country : ", len(un_country))
print("----------"*10)
print("language : ", len(un_language))
print("----------"*10)
print("director : ", len(un_director))
print("----------"*10)
print("release_year : ", len(un_release_year))
print("----------"*10)
print("runtime : ", len(un_runtime))
print("----------"*10)
print("age_rating : ", len(un_age_rating))
print("----------"*10)
print("tags : ", len(un_tags))
print("----------"*10)

storyline :  96
----------------------------------------------------------------------------------------------------
country :  4
----------------------------------------------------------------------------------------------------
language :  0
----------------------------------------------------------------------------------------------------
director :  4
----------------------------------------------------------------------------------------------------
release_year :  25
----------------------------------------------------------------------------------------------------
runtime :  0
----------------------------------------------------------------------------------------------------
age_rating :  121
----------------------------------------------------------------------------------------------------
tags :  98
----------------------------------------------------------------------------------------------------


<hr>

# 여기서부터는 저장 관리 영역입니다

In [182]:
temp_result = movie_dataset.loc[500:1000]
temp_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 500 to 1000
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       501 non-null    int64 
 1   title         501 non-null    object
 2   genres        501 non-null    object
 3   country       498 non-null    object
 4   language      500 non-null    object
 5   director      498 non-null    object
 6   release_year  493 non-null    object
 7   runtime       500 non-null    object
 8   age_rating    486 non-null    object
 9   tag_1         496 non-null    object
 10  tag_2         496 non-null    object
 11  tag_3         495 non-null    object
 12  storyline     495 non-null    object
dtypes: int64(1), object(12)
memory usage: 51.0+ KB


In [185]:
temp_result.to_csv("../../dataset/movies_crawling_sample_500_1000.csv")